In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

import re, os, json, codecs, gc
import csv
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

import warnings
warnings.filterwarnings('ignore')

from keras_bert import load_trained_model_from_checkpoint, Tokenizer
from keras.layers import *
from keras.callbacks import *
from keras.models import Model
from keras.optimizers import Adam
from keras import backend as K
from keras.utils import to_categorical

Using TensorFlow backend.


In [19]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/task3_new_stage2.csv")

train_df = train_df.drop_duplicates(
    subset=['text', 'piclist', 'userGender', 'userFollowCount', 'userFansCount', 'userWeiboCount', 'userLocation',
            'userDescription']).reset_index()
train_size = len(train_df)

train_df['label'].value_counts()

0    17042
1    17010
Name: label, dtype: int64

In [20]:
test_df

,id,text,piclist,userGender,userFollowCount,userFansCount,userWeiboCount,userLocation,userDescription,category
0,98a8deab7f69217558c6d39d7569de1f,中央巡视组/纪检监察组驻进十堰市组长施建明：手机号：13520666543136012440...,b76f9ae026813de11a71ab6ee97d1948.png,男,4899.0,6842.0,1192.0,湖北 十堰,以健康为中心，糊涂一点，潇洒一点，忘记名利，忘记年龄，忘记怨恨，有个伴，有个窝，有点钱，有好友。,政治
1,b828f2306ff0a4572594d141b1f180a9,听屋头的人说，自贡商业银行要倒闭了，排队取钱的人排起龙浪长2自贡·豪斯登堡,c05c3d7dc90ea9522f343aacfdf01e8b.jpg,女,257.0,34.0,26.0,其他,阳光，善良正直，安静，淑女。永远不带给任何人负面情绪，我是正能量。,财经商业
2,ff4fda12a9ab29d00808f04e91ac7409,#西安新闻##魅力大西安#【央视《中国新闻》关注西安多措并举“招贤纳士”吸引中外客商投资】今...,0a671ff09e9884a45a443b0a3895061e.jpg,男,408.0,10981.0,1276.0,陕西,本地资讯博主（西安）,政治
3,40920b0a703c1d4dc10189551f159fd0,女子拒绝强奸，男子霸王硬上弓致生殖器折断，女子被判刑3年！！！还有没有法律，这男的死有余辜吧！！！,8f185307f4460d6adce83d2d2195a73e.jpg,男,682.0,923.0,2689.0,福建 厦门,自爱，爱人，人爱！爱一切众生！！,社会生活
4,1bba2f2251169f77711c323183190c5d,京城四少：内地有不少喜欢和女明星传绯闻的富二代，其中几个年轻帅气的被戏称为“京城四少”，即京...,NaN,男,349.0,1051.0,599.0,陕西 西安,心之所向 身之所往 锲而不舍 成之必然,文体娱乐
...,...,...,...,...,...,...,...,...,...,...
3994,2be63420860457249f8491a442dc77c0,亲爱的灾民们，你们不要着急，直升飞机很忙，再等几天都到了，你们一定坚持，一定要坚强，同胞们！...,7ad0c6b0ddfa99b445ea3391d89b4f8e.jpg,男,3164.0,460100.0,21161.0,河北,社会媒体人，生活搞笑视频收录、牛人推荐、娱乐爆料、爆料视频、合作QQ：625429850 （...,灾难事故
3995,38dd82d1849ec20d35d41a372abb126a,\t\t【男子送猫求爱遭拒寄养宠物店争执后狠摔幼猫】16日，南京一男子买猫送女同事，女方拒养...,47311172e3d995acbb201464f84c6209.jpg,男,473.0,1160000.0,21125.0,北京朝阳区,知名搞笑幽默博主,社会生活
3996,3242cba751e8ceeccc75acb514fbee03,【重磅猛料：川酒剑南春“震荡”】剑南春“股权改制”纷争在持续发酵，近日又有可靠消息称，其内部...,NaN,男,313.0,55311.0,11742.0,其他,换个角度看问题,财经商业
3997,3c1b40f28db2b0ee4f1037522c47efd9,纤维化是肝脏炎症和硬化之间的一个中间过程，是一个摇摆的“中间......抗肝纤维化治疗是转化...,a1fcbb559f8f334d3a642db949d476d8.jpg,女,1083.0,10340.0,4319.0,北京,吴阶平医学基金会肝病医学部：卫生部直属的行业基金会，设立肝病医学部，促进肝病新技术、新疗法的...,医药健康


In [11]:
data = pd.concat([train_df, test_df], ignore_index=True, sort=True)
data['textb'] = data['userLocation'].fillna('没有地址')
data['userDescription'] = "@@" + data['userDescription'].fillna('没有描述')
data['category'] = "@@" + data['category'].fillna('没有分类')

data['piclist'] = data['piclist'].fillna('')
data['pic_size'] = data['piclist'].apply(lambda x: str(len(x.split("\t"))) if x else '0')
data['pic_size'] = "@@图片个数" + data['pic_size']
data['textb'] = data['textb'] + data['category'] + data['pic_size']

for col_name in [['userFollowCount', '用户关注数'], ['userFansCount', '粉丝数'], ['userWeiboCount', '发布微博数']]:
    print(col_name)
    col = col_name[0]
    name = col_name[1]
    data[col] = data[col].fillna(0).map(int)
    name = "@@" + name
    data[col] = name + data[col].map(str)
    data['textb'] = data['textb'] + data[col]

data['textb'] = data['textb'] + data['userDescription']

data[['textb']].head()

['userFollowCount', '用户关注数']
['userFansCount', '粉丝数']
['userWeiboCount', '发布微博数']


,textb
0,北京朝阳区@@文体娱乐@@图片个数0@@用户关注数1728@@粉丝数748@@发布微博数30...
1,云南楚雄@@社会生活@@图片个数1@@用户关注数423@@粉丝数112@@发布微博数792@...
2,没有地址@@社会生活@@图片个数1@@用户关注数0@@粉丝数0@@发布微博数0@@没有描述
3,北京东城区@@社会生活@@图片个数1@@用户关注数1668@@粉丝数7470000@@发布微...
4,江苏盐城@@社会生活@@图片个数0@@用户关注数267@@粉丝数61@@发布微博数1098@...


In [12]:
train_df = data.iloc[:train_size].reset_index(drop=True)
test_df = data.iloc[train_size:].reset_index(drop=True)

In [27]:
data

,category,id,index,label,piclist,text,userDescription,userFansCount,userFollowCount,userGender,userLocation,userWeiboCount,textb,pic_size
0,@@文体娱乐,84cdcfed1aeb7047ad168be7bb9b559e,0.0,0.0,,回复新浪网友对博文【国家文物局限制鉴宝节目现场估价转】的评论：;;查看原文：,@@我们虚度的今天恰是。。昨天去世人渴望的明天。,@@粉丝数748,@@用户关注数1728,男,北京朝阳区,@@发布微博数30884,北京朝阳区@@文体娱乐@@图片个数0@@用户关注数1728@@粉丝数748@@发布微博数30...,@@图片个数0
1,@@社会生活,93ebc056c547618b5b00ab35270c9fad,1.0,0.0,63ad082a189566eed7c4bb3e4bc55012.jpg,//分享网易新闻:《发生在昆明的火锅店老板“辱滇门”，云南人该愤怒还是羞愧》|发生在昆明.....,@@用心生活,@@粉丝数112,@@用户关注数423,男,云南楚雄,@@发布微博数792,云南楚雄@@社会生活@@图片个数1@@用户关注数423@@粉丝数112@@发布微博数792@...,@@图片个数1
2,@@社会生活,eefcba3b5856fe8f55213e036ee463ca,2.0,0.0,4986dc2a5f09a87c7af5dfc57d7775cd.jpg,西宁城管围殴民警扬言要把警察打死|西宁城管围...,@@没有描述,@@粉丝数0,@@用户关注数0,NaN,NaN,@@发布微博数0,没有地址@@社会生活@@图片个数1@@用户关注数0@@粉丝数0@@发布微博数0@@没有描述,@@图片个数1
3,@@社会生活,8e09b1b13477f62139b5cd7a7a7dcb8f,3.0,0.0,dcfccfc69e90a0007afd6aafa1385e56.jpg,【川航航班因驾驶舱风挡破裂安全备降成都】今天上午6:26从重庆江北国际机场出发前往拉萨的四川...,@@中国青年报•中青在线微博,@@粉丝数7470000,@@用户关注数1668,女,北京东城区,@@发布微博数57256,北京东城区@@社会生活@@图片个数1@@用户关注数1668@@粉丝数7470000@@发布微...,@@图片个数1
4,@@社会生活,9ffea4d9573c6e723e8f178a789888dc,4.0,0.0,,支持郑强！！！//【贵州大学校长回应空姐言论:常给她们写感谢信】,@@能烧得全烧了，只剩下石头。,@@粉丝数61,@@用户关注数267,男,江苏盐城,@@发布微博数1098,江苏盐城@@社会生活@@图片个数0@@用户关注数267@@粉丝数61@@发布微博数1098@...,@@图片个数0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38046,@@灾难事故,2be63420860457249f8491a442dc77c0,NaN,NaN,7ad0c6b0ddfa99b445ea3391d89b4f8e.jpg,亲爱的灾民们，你们不要着急，直升飞机很忙，再等几天都到了，你们一定坚持，一定要坚强，同胞们！...,@@社会媒体人，生活搞笑视频收录、牛人推荐、娱乐爆料、爆料视频、合作QQ：625429850...,@@粉丝数460100,@@用户关注数3164,男,河北,@@发布微博数21161,河北@@灾难事故@@图片个数1@@用户关注数3164@@粉丝数460100@@发布微博数21...,@@图片个数1
38047,@@社会生活,38dd82d1849ec20d35d41a372abb126a,NaN,NaN,47311172e3d995acbb201464f84c6209.jpg,\t\t【男子送猫求爱遭拒寄养宠物店争执后狠摔幼猫】16日，南京一男子买猫送女同事，女方拒养...,@@知名搞笑幽默博主,@@粉丝数1160000,@@用户关注数473,男,北京朝阳区,@@发布微博数21125,北京朝阳区@@社会生活@@图片个数1@@用户关注数473@@粉丝数1160000@@发布微博...,@@图片个数1
38048,@@财经商业,3242cba751e8ceeccc75acb514fbee03,NaN,NaN,,【重磅猛料：川酒剑南春“震荡”】剑南春“股权改制”纷争在持续发酵，近日又有可靠消息称，其内部...,@@换个角度看问题,@@粉丝数55311,@@用户关注数313,男,其他,@@发布微博数11742,其他@@财经商业@@图片个数0@@用户关注数313@@粉丝数55311@@发布微博数1174...,@@图片个数0
38049,@@医药健康,3c1b40f28db2b0ee4f1037522c47efd9,NaN,NaN,a1fcbb559f8f334d3a642db949d476d8.jpg,纤维化是肝脏炎症和硬化之间的一个中间过程，是一个摇摆的“中间......抗肝纤维化治疗是转化...,@@吴阶平医学基金会肝病医学部：卫生部直属的行业基金会，设立肝病医学部，促进肝病新技术、新疗...,@@粉丝数10340,@@用户关注数1083,女,北京,@@发布微博数4319,北京@@医药健康@@图片个数1@@用户关注数1083@@粉丝数10340@@发布微博数431...,@@图片个数1


In [4]:
x

array([['回复新浪网友对博文【国家文物局限制鉴宝节目现场估价转】的评论：;;查看原文：'],
       ['//分享网易新闻:《发生在昆明的火锅店老板“辱滇门”，云南人该愤怒还是羞愧》|发生在昆明...@网易新闻客户端#网易新闻#'],
       ['西宁城管围殴民警扬言要把警察打死|西宁城管围...'],
       ...,
       ['妇幼保健院提示您，请不要给宝宝喝爽歪歪和有添加剂的牛奶饮料，告诉家里有小孩的朋友，旺仔牛奶、可口可乐、爽歪歪、娃哈哈AD钙奶、多美滋、雅培、美汁源果粒奶优菠萝味的。都含有肉毒杆菌。现在紧急召回。有没有孩子的都转下！！！[好囧][好囧][好囧]'],
       ['请大家5月13号一定别进影院，大家一起为《贞子》票房为零，做努力！中国人拍的《金陵十三钗》在日本小鬼子票房为零。小日本拍的《贞子》3D将于5月12日在中国大陆上映。而5月12日既是南京大屠杀纪念日。请中国人转起来@作业本@端砚名师钟创荣@侯宁'],
       ['复旦大学的四个日本留学生在昨天下午酒后将一名大一在校女生和来看她她的表妹劫持到留学生宿舍，对二人进行了长达几个小时的奸淫。警方已经将其中三人拘留。校方现在要求在校学生对案件保持沉默！受害女生已经发疯，几次尝试自杀未遂！15岁的小表妹，还在急救室！']],
      dtype=object)

In [5]:
#X_train, X_val, Y_train, Y_val = train_test_split(x, y, test_size = 0.2, random_state = 0)